<a href="https://colab.research.google.com/github/otkata19/competition/blob/main/Akutagawa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

In [ ]:
!apt install aptitude

In [ ]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

In [ ]:
!pip install mecab-python3

In [ ]:
!pip install fasttext

In [ ]:
!find / -name mecabrc

In [ ]:
!cp /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
#MeCab確認(python)
import sys
import MeCab
mecab = MeCab.Tagger ("-Owakati")
text = mecab.parse("すもももももももものうち ")
print(text)

In [ ]:
cd drive/MyDrive/Nishika/Akutagawa/data

In [ ]:
import re

s = "［＃「五」は中見出し］"
t = "《しな》"
result = re.sub(r'\［.+\］', '', s)
result_t = re.sub(r'\《.+\》', '', t)

In [ ]:
result_t

In [ ]:
import csv
csv.field_size_limit(1000000000)

#csv読み込み
csv_train = open("train.csv", "r", errors="", newline="" )
csv_test = open("test.csv", "r", errors="", newline="" )
csv_train_data = csv.reader(csv_train, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
csv_test_data = csv.reader(csv_test, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

In [ ]:
import re

mecab = MeCab.Tagger ("-Owakati") #分かち書き設定

def traindata2list(csv_data):
    list_row = [] #分かち書き結果を保存
    for row in csv_data:
        text_r = row[1]
        text_s = re.sub(r"\s", "", text_r)
        text = mecab.parse(text_s)
        list_row.append((row[2]+'   '+text).rstrip("\n"))
    return list_row

def testdata2list(csv_data):
    list_row = [] #分かち書き結果を保存
    for row in csv_data:
        text_r = row[1]
        text_s = re.sub(r"\s", "", text_r)
        text = mecab.parse(text_s)
        list_row.append((text).rstrip("\n"))
    return list_row

In [ ]:
train_pre = traindata2list(csv_train_data)
test_pre = testdata2list(csv_test_data)

In [ ]:
print(len(train_pre), len(test_pre))

In [ ]:
train_pre[:2]

In [ ]:
path_w = 'train_data.txt' #分かち書き済み学習用データ保存ファイル名
with open(path_w, 'w') as f_out:
    # train_pre[0]は'auther body'なのでtrain_pre[1:]
    for row in train_pre[1:]:
        label, text = row.strip().split('   ')
        f_out.write('__label__{} {}\n'.format(label, text))

In [ ]:
pwd

In [ ]:
# fastTextで学習
import fasttext as ft

#model = ft.train_supervised("train_data.txt", label_prefix='__label__',epoch=1000, loss="hs")
model = ft.train_supervised("train_data.txt")
model.save_model("akutagawa.bin")

In [ ]:
len(test_pre)

In [ ]:
ls

In [ ]:
import pandas as pd

df = pd.read_csv('sample_submission.csv')
df_w = df['writing_id']
print(df_w)

In [ ]:
# test[0]は'body'なのでtest_pre[1:]
classifier = ft.load_model('akutagawa.bin')
ret_list = []
for test_text in test_pre[1:]:
    predict_t = mecab.parse(test_text)
    predict_ak = predict_t.replace("\n"," ")
    ret = classifier.predict(predict_ak)
    ret_list.append(ret[0][0].replace("__label__", ""))

In [ ]:
df_a = pd.DataFrame({'author': ret_list})

In [ ]:
df_wa = pd.concat([df_w, df_a], axis=1)
print(df_wa)

In [ ]:
df_wa.to_csv('submission.csv', index=False)

In [ ]:
def ft_predict(wakati):
    classifier = ft.load_model('akutagawa.bin')
    estimate = classifier.predict(wakati)

In [ ]:
predict

In [ ]:
ft_predict(predict)

In [ ]:
import csv

#csv読み込み
csv_file = open("train_data.csv", "r", errors="", newline="" )
csv_data = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)


mecab = MeCab.Tagger ("-Owakati") #分かち書き設定
list_row = [] #分かち書き結果を保存
path_w = 'train_data.txt' #分かち書き済み学習用データ保存ファイル名


for row in csv_data:
    text = mecab.parse (row[1])
    list_row.append((row[0]+'  '+text).rstrip("\n"))
    
f = open(path_w, 'w')
for x in list_row:
    f.write(str(x) + "\n")
f.close()